In [1]:
import pickle
import praw
#from utils import *
import pandas as pd
import pytz
from collections import deque
#import pymongo
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime
import os
import pandas as pd
import numpy as np
import nltk

In [2]:
subreddit_channel = 'politics'


fetch_creds = pd.read_excel('/Users/katamezo/Documents/Columbia_University /spring_2022/natural_language_processing/hw4/creds.xlsx')

reddit = praw.Reddit(
     client_id=fetch_creds.cliend_id[0],
     client_secret=fetch_creds.client_secret[0],
     user_agent="testscript by u/fakebot3",
     username=fetch_creds.username[0],
     password=fetch_creds.password[0],
     check_for_async=False
 )

In [3]:
#fetch_creds.head()

In [4]:
pca_file = open("/Users/katamezo/Documents/Columbia_University /spring_2022/natural_language_processing/hw4/pca.pkl",'rb')
pca = pickle.load(pca_file)

my_model=pd.read_pickle('/Users/katamezo/Documents/Columbia_University /spring_2022/natural_language_processing/hw4/my_model.pkl')
#pca = pd.read_pickle('/Users/katamezo/Documents/Columbia_University /spring_2022/natural_language_processing/hw4/pca.pkl')
#vectorizer = pd.read_pickle('/Users/katamezo/Documents/Columbia_University /spring_2022/natural_language_processing/hw4/vectorizer.pkl')

vec_file=open('/Users/katamezo/Documents/Columbia_University /spring_2022/natural_language_processing/hw4/vectorizer.pkl', 'rb')
vectorizer= pickle.load(vec_file)

/Users/katamezo/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/katamezo/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [5]:
def conv_time(var):
    tmp_df = pd.DataFrame()
    tmp_df = tmp_df.append(
        {'created_at': var},ignore_index=True)
    tmp_df.created_at = pd.to_datetime(
        tmp_df.created_at, unit='s').dt.tz_localize(
            'utc').dt.tz_convert('US/Eastern') 
    return datetime.fromtimestamp(var).astimezone(pytz.utc)

In [6]:
def word_counter(var):
    tmp_wrd = var.keys()
    if "gme" in list(tmp_wrd):
        print ("gme")
        for word in tmp_wrd:
            tmp_cnt = var[word]
            try:
                the_word_freq_dict[word] = the_word_freq_dict[word] + tmp_cnt
            except:
                the_word_freq_dict[word] = tmp_cnt
                pass
    return 0

def score_sentiment(var):
    senti_score = round(
            sentiment_score_var.polarity_scores(var)['compound'], 4)
    var = var.lower()
    if "democrat" in var:
        the_sentiment_dict['democrat'].pop()
        the_sentiment_dict['democrat'].appendleft(senti_score)
    if "republican" in var:
        the_sentiment_dict['republican'].pop()
        the_sentiment_dict['republican'].appendleft(senti_score)    
    return the_sentiment_dict

def my_stem(var_in):
    from nltk.stem.porter import PorterStemmer
    my_stem = PorterStemmer()
    tmp = [my_stem.stem(word) for word in var_in.split()]
    tmp = ' '.join(tmp)
    return tmp

def my_stop_words(var_in):
    from nltk.corpus import stopwords
    sw = stopwords.words('english')
    tmp = [word for word in var_in.split() if word not in sw]
    tmp = ' '.join(tmp)
    return tmp


def clean_txt(txt_in):
    import re
    clean_str = re.sub("[^A-Za-z]+", " ", txt_in).strip().lower()
    return clean_str

In [7]:
def get_reddit_data(var_in):
    import pandas as pd
    tmp_dict = pd.DataFrame()
    tmp_time = None
    try:
        tmp_dict = tmp_dict.append({"created_at": conv_time(
                                        var_in.created_utc)},
                                    ignore_index=True)
        tmp_time = tmp_dict.created_at[0] 
    except:
        print ("ERROR")
        pass
    
    
    tmp_dict = {'msg_id': str(var_in.id),
                'author': str(var_in.author),
                'body': var_in.body, 'datetime': tmp_time, 
                'clean' : my_stem(my_stop_words(clean_txt(var_in.body)))}

    vector = vectorizer.transform([tmp_dict['clean']])
    pca_value = pca.transform(vector.toarray())
    predict = my_model.predict(pca_value)
    tmp_dict['prediction'] = predict
    tmp_dict["probability"] = my_model.predict_proba(pca_value)
    
    return tmp_dict

In [8]:
text= "I love this homeworks cneiu +!Z"
clean_txt(my_stem(my_stop_words(text)))

'i hate homework cneiu z'

In [9]:
for comment in reddit.subreddit(subreddit_channel).stream.comments():
    tmp_df = get_reddit_data(comment)
    print(tmp_df)
    
    

{'msg_id': 'i4mckoc', 'author': 'thatnameagain', 'body': 'I\'m not saying either of those things. I\'m saying that they should do it because its the right thing to do but should not expect any of the goodwill to translate into a net gain in votes, like everyone on Reddit naively assumes it will. \n\nMy point is that "If Biden forgives student loans democrats will get so many more voters" takes are wrong.', 'datetime': Timestamp('2022-04-13 21:40:37+0000', tz='UTC'), 'clean': 'say either thing say right thing expect goodwil translat net gain vote like everyon reddit naiv assum point biden forgiv student loan democrat get mani voter take wrong', 'prediction': array(['democratic_policies'], dtype=object), 'probability': array([[0.57, 0.43]])}
{'msg_id': 'i4mckua', 'author': 'p6one6', 'body': "Not saying that it's right, but I have more belief in the US to deal with it on our own with our values and our voting than to allow an international bureaucracy that is more likely to be corrupt and

{'msg_id': 'i4mcobk', 'author': 'zeptillian', 'body': 'This is the party that literally campaigns for child grooming pedophiles. Majorly Toadie Gangrene is spreading these lies after literally just finishing up her tour with Matt Gaetz who is currently under federal investigation for child sex trafficking. \n\nHow dumb do you have to be to get outraged at the BS claimed by the fugly troll while completely ignoring her association and support of a literal child sex trafficker?', 'datetime': Timestamp('2022-04-13 21:41:17+0000', tz='UTC'), 'clean': 'parti liter campaign child groom pedophil majorli toadi gangren spread lie liter finish tour matt gaetz current feder investig child sex traffick dumb get outrag bs claim fugli troll complet ignor associ support liter child sex traffick', 'prediction': array(['republican_policies'], dtype=object), 'probability': array([[0.43, 0.57]])}
{'msg_id': 'i4mcoh5', 'author': 'RhymeSpitter3000', 'body': 'If someone determines that they want to talk to 

{'msg_id': 'i4mcstg', 'author': 'MetalGramps', 'body': 'It also establishes an association with the accused and pedophilia that pesists in the minds of a lot of voters even of it is debunked.  People will hear their name and remember the term "child molester."  The GOP knows this and hos little reason not to make baseless accusations.', 'datetime': Timestamp('2022-04-13 21:42:06+0000', tz='UTC'), 'clean': 'also establish associ accus pedophilia pesist mind lot voter even debunk peopl hear name rememb term child molest gop know ho littl reason make baseless accus', 'prediction': array(['democratic_policies'], dtype=object), 'probability': array([[0.51, 0.49]])}
{'msg_id': 'i4mct2f', 'author': 'orxngepeaches', 'body': "But just a few yrs ago they were all screaming that grooming wasn't real when talking about the underage bride laws -_-", 'datetime': Timestamp('2022-04-13 21:42:09+0000', tz='UTC'), 'clean': 'yr ago scream groom real talk underag bride law', 'prediction': array(['republic

{'msg_id': 'i4mcy6f', 'author': 'mopsockets', 'body': 'Pregnant people can commit suicide to get out of this situation. More and more people will start choosing this. And suicide is contagious. Buckle up.', 'datetime': Timestamp('2022-04-13 21:43:06+0000', tz='UTC'), 'clean': 'pregnant peopl commit suicid get situat peopl start choos suicid contagi buckl', 'prediction': array(['republican_policies'], dtype=object), 'probability': array([[0.3, 0.7]])}
{'msg_id': 'i4mcyks', 'author': 'centran', 'body': 'It gets a little more complicated then that answer but works off what you said. Because it\'s hard to go after the wealthy Trump/GOP appointed and directed the IRS to go after "more people". (Which means poorer people). This was too done to clog up and waste IRS time/resources making it harder to properly audit the rich.', 'datetime': Timestamp('2022-04-13 21:43:10+0000', tz='UTC'), 'clean': 'get littl complic answer work said hard go wealthi trump gop appoint direct ir go peopl mean poor

KeyboardInterrupt: 